In [9]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.contrib import rnn

from util.ZigZag import ZigZag, ZigZag0
from util.CalcReturns import CalcReturns

from util.BuildData import TrainingSet_NN_Prices, TrainingSet_NN_Logret
from util.BuildData import TrainingSet_ML_Prices, TrainingSet_ML_Logret

In [10]:
def recurrent_neural_network_model(input):

    layer = {'w' : tf.Variable(tf.random_normal([num_hidden, n_classes])),
             'b' : tf.Variable(tf.random_normal([n_classes]))}
    
    input = tf.unstack(input, timesteps, 1)    
    
    lstm_cell = rnn.BasicLSTMCell(num_hidden)    
    outs, state = rnn.static_rnn(lstm_cell, input, dtype=tf.float32)
    
    output = tf.add(tf.matmul(outs[-1], layer['w']), layer['b'], name='nn')    
    return output

def train_and_save_nn(x_data,y_data):
    x_train = tf.placeholder('float', [None, timesteps, num_input], name='x')
    y_train = tf.placeholder('float', [None, n_classes], name='y')
    
    nn = recurrent_neural_network_model(x_train)
    cost = tf.reduce_mean(tf.abs(tf.subtract(nn, y_train)), name='cost')    
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(cost)   
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)   
    
    correct = tf.equal(tf.argmax(nn,1), y_train), name='correct')
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name = 'accuracy')
    
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        
        session.run(tf.initialize_all_variables())
        
        epoch_loss = 0
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int(len(X_train)/batch_size)):
                
                batch_x = x_data[i*batch_size : (i+1)*batch_size]
                batch_y = y_data[i*batch_size : (i+1)*batch_size]
               
                batch_x = batch_x.reshape((batch_size, timesteps, num_input))
                
                _, c = session.run([optimizer, cost], feed_dict = {x_train: batch_x, y_train: batch_y})
                epoch_loss += c
                
            if epoch % 10 == 0:
                print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
               
        saver.save(session, 'data/fxrn_1.ckpt')        
        print('Last epoch loss: ', epoch_loss)
        
    return nn, correct, accuracy

def test_nn(x_test, y_test):
    with tf.Session() as session:
        saver = tf.train.import_meta_graph('data/fxnn_1.ckpt.meta')
        saver.restore(session, tf.train.latest_checkpoint('data/'))
        
        graph    = tf.get_default_graph()
        correct  = graph.get_tensor_by_name('correct:0')
        accuracy = graph.get_tensor_by_name('accuracy:0')
        
        x = graph.get_tensor_by_name('x:0')
        y = graph.get_tensor_by_name('y:0')

        x_test = x_test.reshape((t_count, timesteps, num_input))
        
        r = tf.cast(correct, 'float').eval({x:x_test, y:y_test})
        
        test_buy  = 0.0
        test_sell = 0.0
        test_hold = 0.0

        correct_buy  = 0.0
        correct_sell = 0.0
        correct_hold = 0.0

        incorrect_buy  = 0.0
        incorrect_sell = 0.0
        incorrect_hold = 0.0
        
        for i in range(len(r)):
          
            if y_test[i][2] == 1:
                test_buy += 1.0;
            elif y_test[i][1] == 1:
                test_hold += 1.0;
            elif y_test[i][0] == 1:
                test_sell += 1.0;
                
            if r[i] == 1:
                if y_test[i][2] == 1:
                    correct_buy += 1.0;
                elif y_test[i][1] == 1:
                    correct_hold += 1.0;
                elif y_test[i][0] == 1:
                    correct_sell += 1.0;
            else:
                if y_test[i][2] == 1:
                    incorrect_buy += 1.0;
                elif y_test[i][1] == 1:
                    incorrect_hold += 1.0;
                elif y_test[i][0] == 1:
                    incorrect_sell += 1.0;
        
        print('Accuracy:',accuracy.eval({x_train:x_test, y_train:y_test}))
        
        print( "Test buy:  ", test_buy  )
        print( "Test sell: ", test_sell )
        print( "Test hold: ", test_hold )

        print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/(test_buy+1))*100 )
        print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/(test_sell+1))*100 )
        print( "Correct hold: ", correct_hold, "Incorrect hold: ", incorrect_hold, "Accuracy: ", (correct_hold/(test_hold+1))*100 )

In [11]:
timesteps  = 1
num_input  = 1
n_classes  = 1
step       = num_input * timesteps

hm_epochs  = 200
num_hidden = 160
batch_size = 32

t_count = 1024

In [16]:
%matplotlib

source = pd.read_csv('EURUSD1.csv', header=0) # source CSV
prices = np.array(source.Close) # close prices

T, S = TrainingSet_ML_Logret(prices = prices, minsize = 0.0004, lag = step, scale = True, ZigZagFunc=ZigZag0)

X_train = T['input'][:-t_count]
Y_train = T['label'][:-t_count]

X_test = T['input'][-t_count:]
Y_test = T['label'][-t_count:]

Using matplotlib backend: TkAgg
Buys: 6034; Sells: 8094; Holds: 239; Count: 14367
